# How to use functions with a knowledge base

This notebook builds on the concepts in the [argument generation](How_to_call_functions_with_chat_models.ipynb) notebook, by creating an agent with access to a knowledge base and two functions that it can call based on the user requirement.

We'll create an agent that uses data from arXiv to answer questions about academic subjects. It has two functions at its disposal:
- **get_articles**: A function that gets arXiv articles on a subject and summarizes them for the user with links.
- **read_article_and_summarize**: This function takes one of the previously searched articles, reads it in its entirety and summarizes the core argument, evidence and conclusions.

This will get you comfortable with a multi-function workflow that can choose from multiple services, and where some of the data from the first function is persisted to be used by the second.

## Walkthrough

This cookbook takes you through the following workflow:

- **Search utilities:** Creating the two functions that access arXiv for answers.
- **Configure Agent:** Building up the Agent behaviour that will assess the need for a function and, if one is required, call that function and present results back to the agent.
- **arXiv conversation:** Put all of this together in live conversation.


In [1]:
!pip install scipy --quiet
!pip install tenacity --quiet
!pip install tiktoken==0.3.3 --quiet
!pip install termcolor --quiet
!pip install openai --quiet
!pip install arxiv --quiet
!pip install pandas --quiet
!pip install PyPDF2 --quiet
!pip install tqdm --quiet

  error: subprocess-exited-with-error
  
  × Building wheel for tiktoken (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [37 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-312
      creating build\lib.win-amd64-cpython-312\tiktoken
      copying tiktoken\core.py -> build\lib.win-amd64-cpython-312\tiktoken
      copying tiktoken\load.py -> build\lib.win-amd64-cpython-312\tiktoken
      copying tiktoken\model.py -> build\lib.win-amd64-cpython-312\tiktoken
      copying tiktoken\registry.py -> build\lib.win-amd64-cpython-312\tiktoken
      copying tiktoken\__init__.py -> build\lib.win-amd64-cpython-312\tiktoken
      creating build\lib.win-amd64-cpython-312\tiktoken_ext
      copying tiktoken_ext\openai_public.py -> build\lib.win-amd64-cpython-312\tiktoken_ext
      running egg_info
      writing tiktoken.egg-info\PKG-INFO
      writing dependency_links to tiktoken.egg-

^C


In [2]:
import os
import arxiv
import ast
import concurrent
import json
import os
import pandas as pd
import tiktoken
from csv import writer
from IPython.display import display, Markdown, Latex
from openai import AzureOpenAI
from PyPDF2 import PdfReader
from scipy import spatial
from tenacity import retry, wait_random_exponential, stop_after_attempt
from tqdm import tqdm
from termcolor import colored

GPT_MODEL = "gpt-4o-cde-aia"
EMBEDDING_MODEL = "text-embedding-ada-002-cde-aia"

client = AzureOpenAI(
    azure_endpoint=os.getenv("OPENAI_API_BASE_2"),
    api_key=os.getenv("OPENAI_API_KEY_2"),
    api_version=os.getenv("OPENAI_API_VERSION_2")
)

## Search utilities

We'll first set up some utilities that will underpin our two functions.

Downloaded papers will be stored in a directory (we use ```./data/papers``` here). We create a file ```arxiv_library.csv``` to store the embeddings and details for downloaded papers to retrieve against using ```retrieve_related_doc_summary```.

In [3]:
directory = './data/papers'

# Check if the directory already exists
if not os.path.exists(directory):
    # If the directory doesn't exist, create it and any necessary intermediate directories
    os.makedirs(directory)
    print(f"Directory '{directory}' created successfully.")
else:
    # If the directory already exists, print a message indicating it
    print(f"Directory '{directory}' already exists.")

Directory './data/papers' already exists.


In [4]:
# Set a directory to store downloaded papers
data_dir = os.path.join(os.curdir, "data", "papers")
paper_dir_filepath = "./data/arxiv_library.csv"

# Generate a blank dataframe where we can store downloaded files
df = pd.DataFrame(list())
df.to_csv(paper_dir_filepath)

In [5]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def embedding_request(text):
    response = client.embeddings.create(input=text, model=EMBEDDING_MODEL)
    return response


@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def get_articles(query, library=paper_dir_filepath, top_k=5):
    """This function gets the top_k articles based on a user's query, sorted by relevance.
    It also downloads the files and stores them in arxiv_library.csv to be retrieved by the read_article_and_summarize.
    """
    client = arxiv.Client()
    search = arxiv.Search(
        query = "quantum",
        max_results = 10,
        sort_by = arxiv.SortCriterion.SubmittedDate
    )
    result_list = []
    for result in client.results(search):
        result_dict = {}
        result_dict.update({"title": result.title})
        result_dict.update({"summary": result.summary})

        # Taking the first url provided
        result_dict.update({"article_url": [x.href for x in result.links][0]})
        result_dict.update({"pdf_url": [x.href for x in result.links][1]})
        result_list.append(result_dict)

        # Store references in library file
        response = embedding_request(text=result.title)
        file_reference = [
            result.title,
            result.download_pdf(data_dir),
            response.data[0].embedding,
        ]

        # Write to file
        with open(library, "a") as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(file_reference)
            f_object.close()
    return result_list


In [6]:
# Test that the search is working
result_output = get_articles("ppo reinforcement learning")
result_output[0]

{'title': 'Core-hole Coherent Spectroscopy in Molecules',
 'summary': 'We study the ultrafast dynamics initiated by a coherent superposition of\ncore-excited states of nitrous oxide molecule. Using high-level\n\\textit{ab-initio} methods, we show that the decoherence caused by the\nelectronic decay and the nuclear dynamics is substantially slower than the\ninduced ultrafast quantum beatings, allowing the system to undergo several\noscillations before it dephases. We propose a proof-of-concept experiment using\nthe harmonic up-conversion scheme available at X-ray free-electron laser\nfacilities to trace the evolution of the created core-excited-state coherence\nthrough a time-resolved X-ray photoelectron spectroscopy.',
 'article_url': 'http://dx.doi.org/10.1103/PhysRevLett.132.263202',
 'pdf_url': 'http://arxiv.org/abs/2406.19387v1'}

In [7]:
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100,
) -> list[str]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = embedding_request(query)
    query_embedding = query_embedding_response.data[0].embedding
    strings_and_relatednesses = [
        (row["filepath"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n]


In [8]:
def read_pdf(filepath):
    """Takes a filepath to a PDF and returns a string of the PDF's contents"""
    # creating a pdf reader object
    reader = PdfReader(filepath)
    pdf_text = ""
    page_number = 0
    for page in reader.pages[0:1]:
        page_number += 1
        pdf_text += page.extract_text() + f"\nPage Number: {page_number}"
    return pdf_text


# Split a text into smaller chunks of size n, preferably ending at the end of a sentence
def create_chunks(text, n, tokenizer):
    """Returns successive n-sized chunks from provided text."""
    tokens = tokenizer.encode(text)
    i = 0
    while i < len(tokens):
        # Find the nearest end of sentence within a range of 0.5 * n and 1.5 * n tokens
        j = min(i + int(1.5 * n), len(tokens))
        while j > i + int(0.5 * n):
            # Decode the tokens and check for full stop or newline
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # If no end of sentence found, use n tokens as the chunk size
        if j == i + int(0.5 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j


def extract_chunk(content, template_prompt):
    """This function applies a prompt to some input content. In this case it returns a summarized chunk of text"""
    prompt = template_prompt + content
    response = client.chat.completions.create(
        model=GPT_MODEL, messages=[{"role": "user", "content": prompt}], temperature=0
    )
    return response.choices[0].message.content


def retrieve_related_doc_summary(query):
    """This function does the following:
    - Reads in the arxiv_library.csv file in including the embeddings
    - Finds the closest file to the user's query
    - Scrapes the text out of the file and chunks it
    - Summarizes each chunk in parallel
    - Does one final summary and returns this to the user"""

    # A prompt to dictate how the recursive summarizations should approach the input paper
    summary_prompt = """Summarize this text from an academic paper. Extract any key points with reasoning.\n\nContent:"""

    # If the library is empty (no searches have been performed yet), we perform one and download the results
    library_df = pd.read_csv(paper_dir_filepath).reset_index()
    if len(library_df) == 0:
        print("No papers searched yet, downloading first.")
        get_articles(query)
        print("Papers downloaded, continuing")
        library_df = pd.read_csv(paper_dir_filepath).reset_index()
    library_df.columns = ["title", "filepath", "embedding"]
    library_df["embedding"] = library_df["embedding"].apply(ast.literal_eval)
    strings = strings_ranked_by_relatedness(query, library_df, top_n=1)
    print("Chunking text from paper")
    pdf_text = read_pdf(strings[0])

    # Initialise tokenizer
    tokenizer = tiktoken.get_encoding("cl100k_base")
    results = ""

    # Chunk up the document into 1500 token chunks
    chunks = create_chunks(pdf_text, 1500, tokenizer)
    text_chunks = [tokenizer.decode(chunk) for chunk in chunks]
    print("Summarizing each chunk of text")

    # Parallel process the summaries
    with concurrent.futures.ThreadPoolExecutor(
        max_workers=len(text_chunks)
    ) as executor:
        futures = [
            executor.submit(extract_chunk, chunk, summary_prompt)
            for chunk in text_chunks
        ]
        with tqdm(total=len(text_chunks)) as pbar:
            for _ in concurrent.futures.as_completed(futures):
                pbar.update(1)
        for future in futures:
            data = future.result()
            results += data

    # Final summary
    print("Summarizing into overall summary")
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {
                "role": "user",
                "content": f"""Write a summary collated from this collection of key points extracted from an academic paper.
                        The summary should highlight the core argument, conclusions and evidence, and answer the user's query.
                        User query: {query}
                        The summary should be structured in bulleted lists following the headings Core Argument, Evidence, and Conclusions.
                        Key points:\n{results}\nSummary:\n""",
            }
        ],
        temperature=0,
    )
    return response.choices[0].message.content

In [9]:
print(retrieve_related_doc_summary("PPO reinforcement learning sequence generation"))

Chunking text from paper
Summarizing each chunk of text


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.98s/it]


Summarizing into overall summary
### Core Argument
- The paper "Enhancing Quantum State Discrimination with Indefinite Causal Order" by Spiros Kechrimparis et al. argues that using protocols based on indefinite causal order, such as the quantum switch and its higher-order generalizations (superswitches), can significantly improve the process of quantum state discrimination in noisy channels.

### Evidence
- **Quantum State Discrimination**:
  - Essential for applications in quantum communication, cryptography, and machine learning.
  - The challenge is due to the non-orthogonality of quantum states, making perfect discrimination impossible.
- **Noisy Channels**:
  - Real-world quantum channels are noisy and often unknown, complicating state discrimination.
  - Previous studies indicate that certain protocols can enhance guessing probability even with noise.
- **Quantum Switch and Superswitches**:
  - Introduction of the quantum switch, a supermap that superposes the sequential action o

## Configure Agent

We'll create our agent in this step, including a ```Conversation``` class to support multiple turns with the API, and some Python functions to enable interaction between the ```ChatCompletion``` API and our knowledge base functions.

In [11]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto",  # auto is default, but we'll be explicit, can be required if it must call one or more tools vía {"type": "function", "function": {"name": "my_function"}}, None if no calls are needed
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [12]:
class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "tool": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

In [13]:
rag_tools = [
    {
        "type": "function",
        "function": {
            "name": "doc_retrievalaugmented",
            "description": """Use this function to retrieve information usefull for you to answer the user question or query.""",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Description of the information required to answer a question in plain text based on the user's question or query. If the user's question or query is too complex this input should be a decomposition of the original user question focused on a specific single piece of information."
                    }
                },
            "required": ["query"]
            }
        }
    }
]

available_tools = {
            "doc_retrievalaugmented": retrieve_related_doc_summary
        }

In [14]:
def chat_completion_with_tool_execution(messages, tools=[None], available_tools=available_tools):
    """This function makes a ChatCompletion API call with the option of adding tools"""
    
    response = chat_completion_request(messages, tools)
    print("first response", response)
    response_message = response.choices[0].message
    messages.append(response_message)
    tool_calls = response_message.tool_calls
    
    if tool_calls:
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_tools[function_name]
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"Tool requested, calling function: "+ str(function_name))
            function_response = function_to_call(
                query=function_args.get("query")
            )
            
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
            
        response = chat_completion_request(messages, tools )
    else:
        print(f"Function not required, responding to user")
        
    return response


## arXiv conversation

Let's put this all together by testing our functions out in conversation.

In [16]:
# Start with a system message
paper_system_message = """You are arXivGPT, a helpful assistant pulls academic papers to answer user questions.
You retrieve the papers summaries to answer the customers questions.
Begin!"""
paper_conversation = Conversation()
paper_conversation.add_message("system", paper_system_message)


In [17]:
# Add a user message
paper_conversation.add_message("user", "Hi, how does PPO reinforcement learning work?")
chat_response = chat_completion_with_tool_execution(
    paper_conversation.conversation_history, tools=rag_tools
)
assistant_message = chat_response.choices[0].message.content
paper_conversation.add_message("assistant", assistant_message)
display(Markdown(assistant_message))


first response ChatCompletion(id='chatcmpl-9fDIN0XqyXgk642NEzYBZbOzSbdRn', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_EG8wX9mC7fGbkCy9VTVipBla', function=Function(arguments='{"query":"How does PPO reinforcement learning work?"}', name='doc_retrievalaugmented'), type='function')]), content_filter_results={})], created=1719609583, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_abc28019ad', usage=CompletionUsage(completion_tokens=24, prompt_tokens=149, total_tokens=173), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])
Tool requested, calling function: doc_ret

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.91s/it]


Summarizing into overall summary


Proximal Policy Optimization (PPO) is a type of reinforcement learning algorithm that improves upon previous methods by balancing ease of implementation and sample efficiency. Here’s a high-level explanation of how PPO works:

1. **Policy Representation**: PPO maintains a policy, typically represented as a neural network, which maps states to actions probabilities. It also maintains a value function to estimate the expected return (value) for each state.

2. **Interaction with the Environment**: The agent interacts with the environment by following the current policy to collect trajectories (sequences of states, actions, and rewards).

3. **Advantage Estimation**: For each state-action pair in the trajectories, PPO estimates the "advantage", which is a measure of how much better the action taken was compared to the average action at that state. This helps in stabilizing learning.

4. **Clipped Surrogate Objective**: PPO modifies the objective function of previous policy gradient methods to include a clipping term. This term ensures that the new policy doesn't deviate too much from the old policy, preventing large and potentially harmful updates. The clipped objective function is:
   \[
   \max\left(\mathbb{E}_t\left[\frac{\pi_{\theta}(a_t|s_t)}{\pi_{\theta_{\text{old}}}(a_t|s_t)} \hat{A}_t\right], \mathbb{E}_t\left[\text{clip}\left(\frac{\pi_{\theta}(a_t|s_t)}{\pi_{\theta_{\text{old}}}(a_t|s_t)}, 1 - \epsilon, 1 + \epsilon\right) \hat{A}_t\right]\right)
   \]
   where \(\hat{A}_t\) is the estimated advantage, \(\pi_{\theta}\) and \(\pi_{\theta_{\text{old}}}\) are the new and old policies, respectively, and \(\epsilon\) is a small hyperparameter that defines the clipping range.

5. **Optimization**: The network parameters are updated using gradient descent, optimizing the clipped surrogate objective. This allows the policy to improve over time while ensuring that updates are not too aggressive (due to the clipping).

6. **Value Function Update**: Alongside, the value function is also updated by minimizing the mean squared error between the predicted value and the observed return.

7. **Epochs and Mini-batches**: Instead of using single updates, PPO uses multiple epochs of mini-batch updates to improve the stability of learning.

In summary, PPO strikes a balance between simplicity and robustness, providing a method that is easier to implement than Trust Region Policy Optimization (TRPO) but more effective than vanilla policy gradients. The core idea is to ensure stable updates by clipping the policy objective and using multiple epochs of mini-batch updates.

In [18]:
# Add another user message to induce our system to use the second tool
paper_conversation.add_message(
    "user",
    "Now, I want to know the difference between PPO reinforcement learning and Quantum contextuality",
)
updated_response = chat_completion_with_tool_execution(
    paper_conversation.conversation_history, tools=rag_tools
)
display(Markdown(updated_response.choices[0].message.content))


first response ChatCompletion(id='chatcmpl-9fDJ11nNQLuDcmsFO3yRb3fL1KKLw', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_EG8wX9mC7fGbkCy9VTVipBla', function=Function(arguments='{"query": "Difference between PPO reinforcement learning and Quantum contextuality"}', name='doc_retrievalaugmented'), type='function'), ChatCompletionMessageToolCall(id='call_vz2egpojISU88GFhMcvg6Vuh', function=Function(arguments='{"query": "What is Quantum contextuality?"}', name='doc_retrievalaugmented'), type='function')]), content_filter_results={})], created=1719609623, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_abc28019ad', usage=CompletionUsage(completion_tokens=65, prompt_tokens=1153, total_tokens=1218), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False,

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.94s/it]


Summarizing into overall summary
Tool requested, calling function: doc_retrievalaugmented
Chunking text from paper
Summarizing each chunk of text


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.90s/it]


Summarizing into overall summary


### Difference between PPO Reinforcement Learning and Quantum Contextuality

#### PPO Reinforcement Learning

**Core Argument**: Proximal Policy Optimization (PPO) is a reinforcement learning algorithm designed to optimize a policy to balance exploration and exploitation effectively. It aims to improve decision-making in uncertain environments by ensuring updates are stable and efficient.

**How It Works**:
- **Policy Representation**: Uses a neural network to map states to action probabilities.
- **Interaction with the Environment**: The agent collects data by interacting with the environment.
- **Advantage Estimation**: Calculates the advantage to measure action quality.
- **Clipped Surrogate Objective**: Uses a modified objective function with a clipping term to ensure updates don't deviate too much from the current policy.
- **Optimization**: Updates the policy using gradient descent on the clipped surrogate objective.
- **Value Function Update**: Minimizes the error in value estimates.
- **Epochs and Mini-Batches**: Utilizes multiple epochs of mini-batch updates for stability.

**Conclusions**: PPO is effective for training agents in complex environments, offering robustness and scalability for reinforcement learning tasks.

#### Quantum Contextuality

**Core Argument**: Quantum contextuality is a fundamental aspect of quantum mechanics where the outcome of a measurement cannot be explained by any non-contextual hidden variable theory. It highlights the non-classical nature of quantum measurements.

**How It Works**:
- **Quantum Measurements**: The outcome depends on the context, meaning the set of other measurements that could be performed alongside it.
- **Experimentation and Models**: Demonstrated through various experiments and theoretical constructs that show measurement outcomes are not independent of other compatible measurements.

**Evidence and Findings**:
- **Quantum State Discrimination**: Techniques like quantum switches and superswitches (higher-order superpositions) can improve quantum state discrimination in noisy channels.
- **Indefinite Causal Order**: Using quantum switches allows superposing the sequence of quantum operations, enhancing performance in tasks like state discrimination.

**Conclusions**: Quantum contextuality is essential to understanding and leveraging the peculiarities of quantum mechanics, with significant implications for quantum computation and information theory.

### Summary

- **Domain**: 
  - PPO Reinforcement Learning belongs to **machine learning**, focusing on optimizing decision-making processes.
  - Quantum Contextuality is a concept in **quantum mechanics**, dealing with the non-classical outcomes of quantum measurements.

- **Problem Type**:
  - PPO addresses policy optimization for agents interacting with uncertain environments.
  - Quantum Contextuality addresses the fundamental nature of quantum measurements and their dependence on measurement context.

Both concepts are pivotal in their respective fields but address very different types of problems—PPO in optimizing machine learning algorithms, and Quantum Contextuality in elucidating the fundamental behaviors of quantum systems.

In [21]:
paper_conversation.conversation_history

[{'role': 'system',
  'content': 'You are arXivGPT, a helpful assistant pulls academic papers to answer user questions.\nYou retrieve the papers summaries to answer the customers questions.\nBegin!'},
 {'role': 'user', 'content': 'Hi, how does PPO reinforcement learning work?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_EG8wX9mC7fGbkCy9VTVipBla', function=Function(arguments='{"query":"How does PPO reinforcement learning work?"}', name='doc_retrievalaugmented'), type='function')]),
 {'tool_call_id': 'call_EG8wX9mC7fGbkCy9VTVipBla',
  'role': 'tool',
  'name': 'doc_retrievalaugmented',
  'content': "### Core Argument\n- The paper presents a quantum algorithm that achieves a nearly quartic speedup for the Planted Noisy kXOR problem, a significant improvement over classical algorithms.\n- The algorithm is resource-efficient, requiring only a logarithmic number of qubits.\n- The approach generalizes and simp